# 洗錢
- 洗錢手法: http://www.fullgoal.com.cn/contents/2016/6/22-3bbe1d7f2c28489abfa1aaf2a1b919e7.html
- 剔除異常數據: https://www.zhihu.com/question/399472607

- dp_df
    - DEBIT是進錢(+) & CREDIT是出錢(-) 

In [112]:
import os
import pandas as pd
import numpy as np 
from itertools import product
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', 500)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [41]:
def display_cust_data(data, cust_id):
    cust_df = data[data['cust_id']==cust_id]
    return cust_df

In [42]:
ccba_df = pd.read_csv('data/dataset1/public_train_x_ccba_full_hashed.csv')
cdtx_df = pd.read_csv('data/dataset1/public_train_x_cdtx0001_full_hashed.csv')
custinfo_df = pd.read_csv('data/dataset1/public_train_x_custinfo_full_hashed.csv')
dp_df = pd.read_csv('data/dataset1/public_train_x_dp_full_hashed.csv')
remit_df = pd.read_csv('data/dataset1/public_train_x_remit1_full_hashed.csv')
tr_alertX_df = pd.read_csv('data/dataset1/train_x_alert_date.csv')
tr_sarY_df = pd.read_csv('data/dataset1/train_y_answer.csv')
public_alertX_df = pd.read_csv('data/dataset1/public_x_alert_date.csv')

**Preprocess**
- 將alert_date, sar_flag merge到custinfo

In [44]:
#mapping alert_date & sar flag
alertDate_dict = tr_alertX_df.set_index('alert_key').to_dict()['date']
sarFlag_dict = tr_sarY_df.set_index('alert_key').to_dict()['sar_flag']
public_alertX_df['sar_flag']=2
alertDatePublc_dict = public_alertX_df.set_index('alert_key').to_dict()['date']
sarFlagPublic_dict = public_alertX_df.set_index('alert_key').to_dict()['sar_flag']
custinfo_df['alert_date'] = custinfo_df['alert_key'].map(lambda x: alertDate_dict[x] if x in alertDate_dict else alertDatePublc_dict[x] )
custinfo_df['sar_flag'] = custinfo_df['alert_key'].map(lambda x: sarFlag_dict[x] if x in sarFlag_dict else sarFlagPublic_dict[x])
custinfo_df['sar_flag_nunique'] = custinfo_df.groupby(['cust_id','alert_date'])['sar_flag'].transform('nunique')
invalid_data_qty = sum(custinfo_df['sar_flag_nunique']>1)
print(f'invalid_data_qty: {invalid_data_qty}')
custinfo_valid = custinfo_df[custinfo_df['sar_flag_nunique']==1]
custinfo_valid.drop_duplicates(['cust_id','alert_date','sar_flag'], inplace=True)
custinfo_valid = custinfo_valid.set_index(['cust_id','alert_date'])


invalid_data_qty: 2


/Users/jayhsu/Library/Python/3.8/lib/python/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


## Feturing

**1. by Date**
1. 把各種資料表轉換為以date為單位的特徵
- ccba: 信用卡相關資料, 以月為單位的資料, 缺少public dataset區間,不需處理
    
- cdtx: 消費細項
    - 每日消費金額
    - 每日消費次數
    - 每日台幣消費金額
    - 每日台幣消費次數   
    - 每日外幣消費金額
    - 每日外幣消費次數
    - 每日國內消費金額
    - 每日國內消費次數    
    - 每日國外消費金額
    - 每日國外消費次數
- dp: 借貸: 金額需要乘上匯率
    - 借貸金額
    - 借貸次數
    - CR金額
    - CR次數
    - DB金額
    - DB次數
    - 臨櫃現金交易金額
    - 臨櫃現金交易次數
    - 非臨櫃現金交易金額
    - 非臨櫃現金交易次數 
    - 跨行交易金額
    - 跨行交易次數
    - 非跨行交易金額
    - 非跨行交易次數
    - 實體ATM交易金額
    - 實體ATM交易次數
    - 非實體ATM交易金額
    - 非實體ATM交易次數  
    - 分行數量
- remit
    - 外匯金額
    - 外匯次數    

In [45]:
#cdtx
cdtxDate_ = cdtx_df.groupby(['cust_id','date']).agg({'amt':sum, 'date':'count'}).rename(columns={'amt':'cdtx_amt', 'date':'cdtx_cnt'})
cdtxDate_ntd = cdtx_df[cdtx_df['cur_type']==47].groupby(['cust_id','date']).agg({'amt':sum, 'date':'count'}).rename(columns={'amt':'cdtx_amtNTD', 'date':'cdtx_cntNTD'})
cdtxDate_fc = cdtx_df[cdtx_df['cur_type']!=47].groupby(['cust_id','date']).agg({'amt':sum, 'date':'count'}).rename(columns={'amt':'cdtx_amtFC', 'date':'cdtx_cntFC'})
cdtxDate_tw = cdtx_df[cdtx_df['country']==130].groupby(['cust_id','date']).agg({'amt':sum, 'date':'count'}).rename(columns={'amt':'cdtx_amtTW', 'date':'cdtx_cntTW'})
cdtxDate_f = cdtx_df[cdtx_df['country']!=130].groupby(['cust_id','date']).agg({'amt':sum, 'date':'count'}).rename(columns={'amt':'cdtx_amtF', 'date':'cdtx_cntF'})
cdtxDate_df = pd.concat([cdtxDate_, cdtxDate_ntd, cdtxDate_fc, cdtxDate_tw, cdtxDate_f], axis=1)

#dp
dp_df['amt'] = dp_df['tx_amt']*dp_df['exchg_rate']
dpDate_ = dp_df.groupby(['cust_id','tx_date']).agg({'tx_amt':sum, 'tx_date':'count'}).rename(columns={'tx_amt':'dp_amt', 'tx_date':'dp_cnt'})
dpDate_CR = dp_df[dp_df['debit_credit']=='CR'].groupby(['cust_id','tx_date']).agg({'tx_amt':sum, 'tx_date':'count'}).rename(columns={'tx_amt':'dp_amtCR', 'tx_date':'dp_cntCR'})
dpDate_DB = dp_df[dp_df['debit_credit']=='DB'].groupby(['cust_id','tx_date']).agg({'tx_amt':sum, 'tx_date':'count'}).rename(columns={'tx_amt':'dp_amtDB', 'tx_date':'dp_cntDB'})
dpDate_CC = dp_df[(dp_df['tx_type']==1)&(dp_df['info_asset_code']==12)].groupby(['cust_id','tx_date']).agg({'tx_amt':sum, 'tx_date':'count'}).rename(columns={'tx_amt':'dp_amtCC', 'tx_date':'dp_cntCC'})
dpDate_NCC = dp_df[~((dp_df['tx_type']==1)&(dp_df['info_asset_code']==12))].groupby(['cust_id','tx_date']).agg({'tx_amt':sum, 'tx_date':'count'}).rename(columns={'tx_amt':'dp_amtNCC', 'tx_date':'dp_cntNCC'})
dpDate_CBank = dp_df[dp_df['cross_bank']==1].groupby(['cust_id','tx_date']).agg({'tx_amt':sum, 'tx_date':'count'}).rename(columns={'tx_amt':'dp_amtCBank', 'tx_date':'dp_cntCBank'})
dpDate_InBank = dp_df[dp_df['cross_bank']==0].groupby(['cust_id','tx_date']).agg({'tx_amt':sum, 'tx_date':'count'}).rename(columns={'tx_amt':'dp_amtInBank', 'tx_date':'dp_cntInBank'})
dpDate_ATM = dp_df[dp_df['ATM']==1].groupby(['cust_id','tx_date']).agg({'tx_amt':sum, 'tx_date':'count'}).rename(columns={'tx_amt':'dp_amtATM', 'tx_date':'dp_cntATM'})
dpDate_NATM = dp_df[dp_df['ATM']==0].groupby(['cust_id','tx_date']).agg({'tx_amt':sum, 'tx_date':'count'}).rename(columns={'tx_amt':'dp_amtNATM', 'tx_date':'dp_cntNATM'})
dpDate_branchNunique = dp_df.groupby(['cust_id','tx_date']).agg({'txbranch':'nunique', }).rename(columns={'txbranch':'dp_branchNunique'})
dpDate_df = pd.concat([dpDate_, dpDate_CR, dpDate_DB, dpDate_CC, dpDate_NCC, dpDate_CBank, dpDate_InBank, dpDate_ATM, dpDate_NATM, dpDate_branchNunique], axis=1)

#remit
remitDate_df = remit_df.groupby(['cust_id','trans_date']).agg({'trade_amount_usd':sum, 'trans_no':'count'}).rename(columns={'trade_amount_usd':'remit_amt', 'trans_no':'remit_cnt'})


**2. Accumulate recent K days**   
- accumulate feature: 整合前K天的feature
- Process: 每一個customer要分別做以下的feature engineering
    - 1. 組合cust_id * dates
    - 2. merge with date feature
    - 3. calculate accum feature
    - 4. merge date and accum features


In [61]:
def get_accum_feature(cust_data, cust_id, date_colName, rolling_window):
    dates = list(range(cust_data.index.min(), cust_data.index.max()+1))
    all_custDate = pd.DataFrame(list(product([cust_id], dates)), columns=['cust_id', date_colName]).set_index(['cust_id', date_colName])
    cust_fullDate = pd.merge(cust_data, all_custDate, how='outer', left_index=True, right_index=True)
    cust_fullDate.sort_index(ascending=True, inplace=True)
    cust_recentAccum = cust_fullDate.rolling(window = rolling_window).sum().shift()
    new_cols = []
    for c in cust_recentAccum.columns:
        new_cols.append(c+'_Accum')
    cust_recentAccum.columns=new_cols
    feature = pd.merge(cust_data, cust_recentAccum, how='left', left_index=True, right_index=True)
    return feature

def accum_featuring(dateFeature, date_colName='date', rolling_window=5):
    idx_custid, _ = zip(*dateFeature.index)
    idx_custid = np.unique(idx_custid)
    accu_feature_list = Parallel(n_jobs=4)(delayed(get_accum_feature)(dateFeature.loc[(cust_id)], cust_id, date_colName, rolling_window) for cust_id in idx_custid) 
    accu_feature = pd.concat(accu_feature_list)
    return accu_feature




In [361]:
%%time
cdtx_feature = accum_featuring(cdtxDate_df, 'date', 30)
dp_feature = accum_featuring(dpDate_df, 'tx_date', 30)
remit_feature = accum_featuring(remitDate_df, 'trans_date', 30)

CPU times: user 28.6 s, sys: 1.39 s, total: 30 s
Wall time: 32.1 s


**3. Integrate dataset**
- moneyLundry dataset (sar_flag==1, in alert_date)
- falseAlarm dataset (sar_flag==0, in alert_date)
- public dataset (sar_flag==2, in alert_date)

In [362]:
data_df = pd.concat([custinfo_valid, cdtx_feature, dp_feature, remit_feature], axis=1)
data_df = data_df[data_df['alert_key'].notna()]

In [433]:
labelY = 'sar_flag'
featureX = ['risk_rank', 'occupation_code', 'total_asset', 'AGE',
       'cdtx_amt', 'cdtx_cnt', 'cdtx_amtNTD',
       'cdtx_cntNTD', 'cdtx_amtFC', 'cdtx_cntFC', 'cdtx_amtTW', 'cdtx_cntTW',
       'cdtx_amtF', 'cdtx_cntF', 'cdtx_amt_Accum', 'cdtx_cnt_Accum',
       'cdtx_amtNTD_Accum', 'cdtx_cntNTD_Accum', 'cdtx_amtFC_Accum',
       'cdtx_cntFC_Accum', 'cdtx_amtTW_Accum', 'cdtx_cntTW_Accum',
       'cdtx_amtF_Accum', 'cdtx_cntF_Accum', 'dp_amt', 'dp_cnt', 'dp_amtCR',
       'dp_cntCR', 'dp_amtDB', 'dp_cntDB', 'dp_amtCC', 'dp_cntCC', 'dp_amtNCC',
       'dp_cntNCC', 'dp_amtCBank', 'dp_cntCBank', 'dp_amtInBank',
       'dp_cntInBank', 'dp_amtATM', 'dp_cntATM', 'dp_amtNATM', 'dp_cntNATM',
       'dp_branchNunique', 'dp_amt_Accum', 'dp_cnt_Accum', 'dp_amtCR_Accum',
       'dp_cntCR_Accum', 'dp_amtDB_Accum', 'dp_cntDB_Accum', 'dp_amtCC_Accum',
       'dp_cntCC_Accum', 'dp_amtNCC_Accum', 'dp_cntNCC_Accum',
       'dp_amtCBank_Accum', 'dp_cntCBank_Accum', 'dp_amtInBank_Accum',
       'dp_cntInBank_Accum', 'dp_amtATM_Accum', 'dp_cntATM_Accum',
       'dp_amtNATM_Accum', 'dp_cntNATM_Accum', 'dp_branchNunique_Accum',
       'remit_amt', 'remit_cnt', 'remit_amt_Accum', 'remit_cnt_Accum']

data_df[labelY].value_counts().to_frame()

,sar_flag
0.0,23451
2.0,1770
1.0,233


## Model Training

In [454]:
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from collections import namedtuple
SCORE_TUPLE = namedtuple('SCORE_TUPLE', 'model, iter, precision')

In [434]:

def model_training(model, X_tr, y_tr, preprocess_scale=True, preprocess_pca=True):
    if preprocess_scale:
        scaler = MinMaxScaler().fit(X_tr)
        X_tr = scaler.transform(X_tr)
    else:
        scaler=None
        
    if preprocess_pca:
        pca = PCA(n_components=0.85)
        pca.fit(X_trScaled)
        X_tr = pca.transform(X_tr)
    else:
        pca=None
    
    X_re, y_re = SMOTE().fit_resample(X_tr, y_tr)
    model = model.fit(X_re, y_re)
    return model, scaler, pca 

def predict_prob(model, X_val, scaler, pca, preprocess_scale=True, preprocess_pca=True):
    if preprocess_scale:
        X_val = scaler.transform(X_val)
    if preprocess_pca:
        X_val = pca.transform(X_val)
    y_predProb = model.predict_proba(X_val)
    y_predProb = [x[1] for x in y_predProb]
    return y_predProb

def get_recallN_Precision(y_predProb, y_true):
    y_pred_df = pd.DataFrame(list(zip(y_predProb, y_true)), columns=['predProb','trueLabel'])
    y_pred_df.sort_values(by='predProb', ascending=False, inplace=True)
    y_pred_df['idx']=list(range(len(y_pred_df)))
    idx = y_pred_df[y_pred_df['trueLabel']==1]['idx'].iloc[-2]
    precision_score = sum(y_true)/idx
    return precision_score

In [ ]:
dataset_Tr = data_df[data_df['sar_flag']!=2]
dataset_TsPub = data_df[data_df['sar_flag']==2]

**KFOLD**

In [464]:
%%time
preprocess_scale=True
preprocess_pca=True
X = dataset_Tr[featureX].fillna(0)
Y = dataset_Tr[labelY]
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=23)
skf.get_n_splits(X,Y)
score_list = []

for i, (tr_idx, val_idx) in enumerate(skf.split(X, Y)):
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = Y.iloc[tr_idx], Y.iloc[val_idx]
    
    modelLR = LogisticRegression(max_iter=300)
    modelXGBC = XGBClassifier(use_label_encoder=False)
    modelMLPC = MLPClassifier(max_iter=500)
    
    pred_prob_list = []
    for model in [modelLR, modelXGBC, modelMLPC]:
        model, scaler, pca = model_training(model, X_tr.copy(), y_tr.copy(), preprocess_scale, preprocess_pca)
        y_predProb = predict_prob(model, X_val.copy(), scaler, pca, preprocess_scale, preprocess_pca)
        precisionScore = get_recallN_Precision(y_predProb, y_val)
        score_list.append(SCORE_TUPLE(model.__class__.__name__, i, precisionScore))
        pred_prob_list.append(y_predProb)
        
    precisionScore = get_recallN_Precision(np.mean(pred_prob_list, axis=0), y_val)
    score_list.append(SCORE_TUPLE('ensemble', i, precisionScore))
        


[10:43:29] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:44:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:45:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:46:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Users/jayhsu/Library/Python/3.8/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [465]:
score_df = pd.DataFrame(score_list)
display(score_df.groupby('model')['precision'].mean().to_frame())
display(score_df.groupby(['model','iter'])['precision'].max().to_frame())
display(score_df.groupby(['iter','model'])['precision'].max().to_frame())

,precision
model,
LogisticRegression,0.012175
MLPClassifier,0.011573
XGBClassifier,0.011467
ensemble,0.014021


precision
model              iter           
LogisticRegression 0      0.012091
                   1      0.010779
                   2      0.011444
                   3      0.014387
MLPClassifier      0      0.010246
                   1      0.010567
                   2      0.011522
                   3      0.013958
XGBClassifier      0      0.010038
                   1      0.010833
                   2      0.011350
                   3      0.013648
ensemble           0      0.011640
                   1      0.010625
                   2      0.011781
                   3      0.022040

precision
iter model                        
0    LogisticRegression   0.012091
     MLPClassifier        0.010246
     XGBClassifier        0.010038
     ensemble             0.011640
1    LogisticRegression   0.010779
     MLPClassifier        0.010567
     XGBClassifier        0.010833
     ensemble             0.010625
2    LogisticRegression   0.011444
     MLPClassifier        0.011522
     XGBClassifier        0.011350
     ensemble             0.011781
3    LogisticRegression   0.014387
     MLPClassifier        0.013958
     XGBClassifier        0.013648
     ensemble             0.022040